In [0]:
%fs ls dbfs:/FileStore/shared_uploads/online_retail/

path,name,size
dbfs:/FileStore/shared_uploads/online_retail/online_retail.csv,online_retail.csv,45049428
dbfs:/FileStore/shared_uploads/online_retail/online_retail_II.csv,online_retail_II.csv,43862548


In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType, StringType, DoubleType

schema = (StructType()
            .add("InvoiceNo", StringType(), True)
            .add("StockCode", StringType(), True)
            .add("Description", StringType(), True)
            .add("Quantity", StringType(), True)
            .add("InvoiceDate", StringType(), True)
            .add("UnitPrice", StringType(), True)
            .add("CustomerID", StringType(), True)
            .add("Country", StringType(), True)
         )

df1 = (spark.read
               .option("header", True)
             .csv("dbfs:/FileStore/shared_uploads/online_retail/online_retail.csv"))
df2 = (spark.read
               .option("header", True)
             .csv("dbfs:/FileStore/shared_uploads/online_retail/online_retail_II.csv"))
df1.printSchema()
df2.printSchema()

root
-- InvoiceNo: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: string (nullable = true)
-- InvoiceDate: string (nullable = true)
-- UnitPrice: string (nullable = true)
-- CustomerID: string (nullable = true)
-- Country: string (nullable = true)

root
-- Invoice: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: string (nullable = true)
-- InvoiceDate: string (nullable = true)
-- Price: string (nullable = true)
-- Customer ID: string (nullable = true)
-- Country: string (nullable = true)

In [0]:
retail_df = df1.union(df2)
retail_df.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
 536365| 85123A|WHITE HANGING HEA...| 6|01/12/10 08:26| 2.55| 17850|United Kingdom|
 536365| 71053| WHITE METAL LANTERN| 6|01/12/10 08:26| 3.39| 17850|United Kingdom|
 536365| 84406B|CREAM CUPID HEART...| 8|01/12/10 08:26| 2.75| 17850|United Kingdom|
 536365| 84029G|KNITTED UNION FLA...| 6|01/12/10 08:26| 3.39| 17850|United Kingdom|
 536365| 84029E|RED WOOLLY HOTTIE...| 6|01/12/10 08:26| 3.39| 17850|United Kingdom|
 536365| 22752|SET 7 BABUSHKA NE...| 2|01/12/10 08:26| 7.65| 17850|United Kingdom|
 536365| 21730|GLASS STAR FROSTE...| 6|01/12/10 08:26| 4.25| 17850|United Kingdom|
 536366| 22633|HAND WARMER UNION...| 6|01/12/10 08:28| 1.85| 17850|United Kingdom|
 536366| 22632|HAND WARMER RED P...| 6|01/12/10 08:28| 1.85| 17850|United Kingdom|
 536367| 84879|ASSORTED COLOUR B...| 32|01/12/10 08:34| 1.69| 13047|United Kingdom|
 536367| 22745|POPPY'S PLAYHOUSE...| 6|01/12/10 08:34| 2.1| 13047|United Kingdom|
 536367| 22748|POPPY'S PLAYHOUSE...| 6|01/12/10 08:34| 2.1| 13047|United Kingdom|
 536367| 22749|FELTCRAFT PRINCES...| 8|01/12/10 08:34| 3.75| 13047|United Kingdom|
 536367| 22310|IVORY KNITTED MUG...| 6|01/12/10 08:34| 1.65| 13047|United Kingdom|
 536367| 84969|BOX OF 6 ASSORTED...| 6|01/12/10 08:34| 4.25| 13047|United Kingdom|
 536367| 22623|BOX OF VINTAGE JI...| 3|01/12/10 08:34| 4.95| 13047|United Kingdom|
 536367| 22622|BOX OF VINTAGE AL...| 2|01/12/10 08:34| 9.95| 13047|United Kingdom|
 536367| 21754|HOME BUILDING BLO...| 3|01/12/10 08:34| 5.95| 13047|United Kingdom|
 536367| 21755|LOVE BUILDING BLO...| 3|01/12/10 08:34| 5.95| 13047|United Kingdom|
 536367| 21777|RECIPE BOX WITH M...| 4|01/12/10 08:34| 7.95| 13047|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 20 rows

In [0]:
df3 = (spark.read
             .option("header", True)
             .option("delimiter", ";")
            .csv("/FileStore/shared_uploads/countries_codes.csv"))

country_df = (df3
   .withColumnRenamed("OFFICIAL LANG CODE", "CountryCode")
   .withColumnRenamed("ISO2 CODE", "ISO2Code")
   .withColumnRenamed("ISO3 CODE", "ISO3Code")
   .withColumnRenamed("LABEL EN", "CountryName")
   .withColumnRenamed("Geo Shape", "GeoShape")
   .drop("ONU CODE")
   .drop("IS ILOMEMBER")
   .drop("IS RECEIVING QUEST")
   .drop("LABEL FR")
   .drop("LABEL SP")
   .drop("geo_point_2d")
)

integrated_df = retail_df.join(country_df, retail_df.Country == country_df.CountryName, "left_outer")

In [0]:
country_df.show()
integrated_df.show()

+-----------+--------+--------+--------------------+--------------------+
CountryCode|ISO2Code|ISO3Code| CountryName| GeoShape|
+-----------+--------+--------+--------------------+--------------------+
 EN| CY| CYP| Cyprus|"{""type"": ""Mul...|
 EN| E1| ET1|Ethiopia (includi...| null|
 EN| SS| SSD|South Sudan, The ...| null|
 EN| TR| TUR| Turkey|"{""type"": ""Mul...|
 EN| CN| CHN| China|"{""type"": ""Mul...|
 EN| LY| LBY|Libyan Arab Jamah...|"{""type"": ""Pol...|
 FR| GW| GNB| Guinea-Bissau|"{""type"": ""Mul...|
 EN| MW| MWI| Malawi|"{""type"": ""Mul...|
 EN| BH| BHR| Bahrain|"{""type"": ""Pol...|
 SP| PE| PER| Peru|"{""type"": ""Mul...|
 EN| PH| PHL| Philippines|"{""type"": ""Mul...|
 EN| FJ| FJI| Fiji|"{""type"": ""Mul...|
 EN| JG| CHA| Channel Islands| null|
 FR| RO| ROU| Romania|"{""type"": ""Pol...|
 EN| BS| BHS| Bahamas|"{""type"": ""Mul...|
 EN| GL| GRL| Greenland|"{""type"": ""Mul...|
 EN| BT| BTN| Bhutan|"{""type"": ""Pol...|
 EN| CA| CAN| Canada|"{""type"": ""Mul...|
 EN| JO| JOR| Jordan|"{""type"": ""Pol...|
 EN| PW| PLW| Palau|"{""type"": ""Mul...|
+-----------+--------+--------+--------------------+--------------------+
only showing top 20 rows

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------+--------+--------+--------------+--------------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|CountryCode|ISO2Code|ISO3Code| CountryName| GeoShape|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------+--------+--------+--------------+--------------------+
 536365| 85123A|WHITE HANGING HEA...| 6|01/12/10 08:26| 2.55| 17850|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536365| 71053| WHITE METAL LANTERN| 6|01/12/10 08:26| 3.39| 17850|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536365| 84406B|CREAM CUPID HEART...| 8|01/12/10 08:26| 2.75| 17850|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536365| 84029G|KNITTED UNION FLA...| 6|01/12/10 08:26| 3.39| 17850|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536365| 84029E|RED WOOLLY HOTTIE...| 6|01/12/10 08:26| 3.39| 17850|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536365| 22752|SET 7 BABUSHKA NE...| 2|01/12/10 08:26| 7.65| 17850|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536365| 21730|GLASS STAR FROSTE...| 6|01/12/10 08:26| 4.25| 17850|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536366| 22633|HAND WARMER UNION...| 6|01/12/10 08:28| 1.85| 17850|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536366| 22632|HAND WARMER RED P...| 6|01/12/10 08:28| 1.85| 17850|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536367| 84879|ASSORTED COLOUR B...| 32|01/12/10 08:34| 1.69| 13047|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536367| 22745|POPPY'S PLAYHOUSE...| 6|01/12/10 08:34| 2.1| 13047|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536367| 22748|POPPY'S PLAYHOUSE...| 6|01/12/10 08:34| 2.1| 13047|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536367| 22749|FELTCRAFT PRINCES...| 8|01/12/10 08:34| 3.75| 13047|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536367| 22310|IVORY KNITTED MUG...| 6|01/12/10 08:34| 1.65| 13047|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536367| 84969|BOX OF 6 ASSORTED...| 6|01/12/10 08:34| 4.25| 13047|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536367| 22623|BOX OF VINTAGE JI...| 3|01/12/10 08:34| 4.95| 13047|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536367| 22622|BOX OF VINTAGE AL...| 2|01/12/10 08:34| 9.95| 13047|United Kingdom| EN| GB| GBR|United Kingdom|"{""type"": ""Mul...|
 536367| 21754|HOME BUILDING BLO...| 3|01/12/10 08:34| 5.95| 13047|United Kingdom| EN| GB| GBR|Unit

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType, StringType, DoubleType

schema = (StructType()
            .add("age", StringType(), True)
            .add("workclass", StringType(), True)
            .add("fnlwgt", StringType(), True)
            .add("education", StringType(), True)
            .add("education-num", StringType(), True)
            .add("marital-status", StringType(), True)
            .add("occupation", StringType(), True)
            .add("relationship", StringType(), True)
            .add("race", StringType(), True)
            .add("gender", StringType(), True)
            .add("capital-gain", StringType(), True)
            .add("capital-loss", StringType(), True)
            .add("hours-per-week", StringType(), True)
            .add("native-country", StringType(), True)
         )

In [0]:
#%fs rm -r /FileStore/shared_uploads/delta/retail_enriched.delta

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

income_df = spark.read.schema(schema).csv("/FileStore/shared_uploads/adult.data").withColumn("idx", monotonically_increasing_id())

retail_dfx = integrated_df.withColumn("RetailIDx", monotonically_increasing_id())
income_dfx = income_df.withColumn("IncomeIDx", monotonically_increasing_id()) 
retail_enriched_df = retail_dfx.join(income_dfx, retail_dfx.RetailIDx == income_dfx.IncomeIDx, "left_outer")

display(retail_enriched_df)

InvoiceNo StockCode Description Quantity InvoiceDate UnitPrice CustomerID Country CountryCode ISO2Code ISO3Code CountryName GeoShape RetailIDx age workclass fnlwgt education education-num marital-status occupation relationship race gender capital-gain capital-loss hours-per-week native-country idx IncomeIDx 536365 85123A WHITE HANGING HEART T-LIGHT HOLDER 6 01/12/10 08:26 2.55 17850 United Kingdom EN GB GBR United Kingdom "{""type"": ""MultiPolygon"", ""coordinates"": [[[[-6.287505663, 49.91400788], [-6.309152799, 49.913641669], [-6.298817512, 49.935492255], [-6.284169075, 49.922756252], [-6.287505663, 49.91400788]]], [[[-6.326486783, 49.94920482], [-6.332508918, 49.941717841], [-6.341216601, 49.944769598], [-6.34894772, 49.964341539], [-6.328033007, 49.956610419], [-6.326486783, 49.94920482]]], [[[-1.295765754, 50.773179429], [-1.144276496, 50.733791408], [-1.109120246, 50.732855536], [-1.097035286, 50.726955471], [-1.097035286, 50.713324286], [-1.082753059, 50.713324286], [-1.062326627, 50.692816473], [-1.062326627, 50.685288804], [-1.128651496, 50.666449286], [-1.156361457, 50.650783596], [-1.164702929, 50.640936591], [-1.168446418, 50.603461005], [-1.2792048820000002, 50.582505601], [-1.315541145, 50.589178778], [-1.485951301, 50.669338283], [-1.535145637, 50.669378973], [-1.5694880850000001, 50.658026434], [-1.5453181630000001, 50.686102606], [-1.505930142, 50.707709052], [-1.418690559, 50.733791408], [-1.425363736, 50.720119533], [-1.400868293, 50.721991278], [-1.343495246, 50.761053778], [-1.295765754, 50.773179429]]], [[[-0.9388321610000001, 50.788397528], [-0.9388321610000001, 50.780951239], [-1.02074134, 50.788397528], [-1.02074134, 50.795152085], [-0.998687304, 50.80101146], [-0.972564257, 50.829779364], [-0.9598689440000001, 50.829331773], [-0.947824674, 50.817816473], [-0.9598689440000001, 50.802069403], [-0.9388321610000001, 50.788397528]]], [[[-4.663482226, 51.164862372], [-4.670480924, 51.16010163], [-4.678537564, 51.167059637], [-4.671213345, 51.195054429], [-4.663482226, 51.164862372]]], [[[0.93637129, 51.37103913], [0.9114689460000001, 51.364406643], [0.794769727, 51.370550848], [0.757578972, 51.386419989], [0.7338973320000001, 51.412176825], [0.7468367850000001, 51.439642645], [0.7669376960000001, 51.447943427], [0.9005639980000001, 51.417547919], [0.93246504, 51.397691148], [0.9421492850000001, 51.38300202], [0.93637129, 51.37103913]]], [[[-4.655669726, 53.319769598], [-4.62629147, 53.320868231], [-4.590443489, 53.297552802], [-4.558461067, 53.252752997], [-4.56362871, 53.241929429], [-4.602406379, 53.253078518], [-4.632557746, 53.282945054], [-4.676665819, 53.28510163], [-4.693959114, 53.302801825], [-4.693959114, 53.309637762], [-4.667307095, 53.323919989], [-4.655669726, 53.319769598]]], [[[-4.203236457, 53.298488674], [-4.142486132, 53.302801825], [-4.121978319, 53.317084052], [-4.042876757, 53.302801825], [-4.098052538, 53.25014883], [-4.195546028, 53.220282294], [-4.220366991, 53.201483466], [-4.222035286, 53.182114976], [-4.280140754, 53.154974677], [-4.354359504, 53.129950262], [-4.392404752, 53.138332424], [-4.412831184, 53.132066148], [-4.411976692, 53.155585028], [-4.385121223, 53.182440497], [-4.410878059, 53.182196356], [-4.447580533, 53.159409898], [-4.48167884, 53.17987702], [-4.495350715, 53.17987702], [-4.499623176, 53.211249091], [-4.571034309, 53.275458075], [-4.558664517, 53.3633487], [-4.571034309, 53.392767645], [-4.425363736, 53.426703192], [-4.324696418, 53.419501044], [-4.276722786, 53.401027736], [-4.26691647, 53.368963934], [-4.234730598, 53.358058986], [-4.235178189, 53.34007396], [-4.214833137, 53.323919989], [-4.203236457, 53.298488674]]], [[[-6.440012174, 55.236029364], [-6.415923632, 55.231878973], [-6.378163215, 55.24624258], [-6.239816861, 55.208319403], [-6.160023567, 55.221909898], [-6.120838996, 55.217840887], [-6.047230598, 55.178534247], [-6.037098762, 55.16791413], [-6.040638801, 55.112046617], [-6.062082486, 55.07412344], [-6.054351366, 55.065130927], [-6.003529426,

In [0]:
'''(retail_enriched_df
   .coalesce(1)
   .write
     .format("delta")
     .mode("overwrite")
    .save("/FileStore/shared_uploads/retail.delta"))
    '''

Out[25]: '(retail_enriched_df\n .coalesce(1)\n .write\n .format("delta")\n .mode("overwrite")\n .save("/FileStore/shared_uploads/retail.delta"))\n '

###Data Federation

MySQL Source

In [0]:
dataframe_mysql = spark.read.format("jdbc").options(
    url="jdbc:mysql://localhost:3306/pysparkdb",
    driver = "org.mariadb.jdbc.Driver",
    dbtable = "authors",
    user="root",
    password="root").load()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3486565875271713> in <module> 
 4 dbtable = "authors" , 
 5 user = "root" , 
 ----> 6 password="root").load()
 
 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 182 return self . _df ( self . _jreader . load ( self . _spark . _sc . _jvm . PythonUtils . toSeq ( path ) ) ) 
 183 else : 
 --> 184 return self . _df ( self . _jreader . load ( ) ) 
 185 
 186 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o1978.load.
: java.sql.SQLNonTransientConnectionException: Could not connect to address=(host=localhost)(port=3306)(type=master) : Connection refused (Connection refused)
	at org.mariadb.jdbc.internal.util.exceptions.ExceptionMapper.get(ExceptionMapper.java:161)
	at org.mariadb.jdbc.internal.util.exceptions.ExceptionMapper.connException(ExceptionMapper.java:79)
	at org.mariadb.jdbc.internal.protocol.AbstractConnectProtocol.connectWithoutProxy(AbstractConnectProtocol.java:1040)
	at org.mariadb.jdbc.internal.util.Utils.retrieveProxy(Utils.java:490)
	at org.mariadb.jdbc.MariaDbConnection.newConnection(MariaDbConnection.java:144)
	at org.mariadb.jdbc.Driver.connect(Driver.java:90)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$createConnectionFactory$1(JdbcUtils.scala:64)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:56)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRelation$.getSchema(JDBCRelation.scala:226)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:35)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:387)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:384)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:373)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:373)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:258)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.conne

In [0]:
%sql
CREATE TABLE mysql_authors IF NOT EXISTS
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://localhost:3306/pysparkdb",
  dbtable "authors",
  user "root",
  password "root"
);

SELECT * FROM mysql_authors

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.catalyst.parser.ParseException: 
mismatched input 'IF' expecting { , ';'}(line 1, pos 27)

== SQL ==
CREATE TABLE mysql_authors IF NOT EXISTS
---------------------------^^^
USING org.apache.spark.sql.jdbc
OPTIONS (
 url "jdbc:mysql://localhost:3306/pysparkdb",
 dbtable "authors",
 user "root",
 password "root"
)

	at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(ParseDriver.scala:273)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parse(ParseDriver.scala:142)
	at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:61)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parsePlan(ParseDriver.scala:90)
	at com.databricks.sql.parser.DatabricksSqlParser.$anonfun$parsePlan$1(DatabricksSqlParser.scala:70)
	at com.databricks.sql.parser.DatabricksSqlParser.parse(DatabricksSqlParser.scala:90)
	at com.databricks.sql.parser.DatabricksSqlParser.parsePlan(DatabricksSqlParser.scala:67)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$2(SparkSession.scala:674)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:136)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:674)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:841)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:671)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:672)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:91)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:231)
	at scala.collection.immutable.List.map(List.scala:298)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:37)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:144)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$11(DriverLocal.scala:488)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:235)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:232)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:50)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:277)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:270)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:50)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:465)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:690)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:682)
	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:523)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:635)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:428)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:371)
	at com.databricks.backend.daemon.driver.DriverWrapper.run(DriverWrapper.scala:223)
	at java.lang.Thread.run(Thread.java:748)

	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:129)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:144)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$exec

In [0]:
from pyspark.sql.functions import rand, col

authors_df = spark.range(16).withColumn("salary", rand(10)*col("id")*10000)
authors_df.write.format("csv").saveAsTable("author_salary")

In [0]:
%sql
SELECT
  m.last_name,
  m.first_name,
  s.salary
FROM
  author_salary s
  JOIN mysql_authors m ON m.uid = s.id
ORDER BY s.salary DESC

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: java.util.concurrent.ExecutionException: java.sql.SQLNonTransientConnectionException: Could not connect to address=(host=localhost)(port=3306)(type=master) : Connection refused (Connection refused)
	at com.google.common.util.concurrent.AbstractFuture$Sync.getValue(AbstractFuture.java:299)
	at com.google.common.util.concurrent.AbstractFuture$Sync.get(AbstractFuture.java:286)
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:116)
	at com.google.common.util.concurrent.Uninterruptibles.getUninterruptibly(Uninterruptibles.java:135)
	at com.google.common.cache.LocalCache$Segment.getAndRecordStats(LocalCache.java:2344)
	at com.google.common.cache.LocalCache$Segment.loadSync(LocalCache.java:2316)
	at com.google.common.cache.LocalCache$Segment.lockedGetOrLoad(LocalCache.java:2278)
	at com.google.common.cache.LocalCache$Segment.get(LocalCache.java:2193)
	at com.google.common.cache.LocalCache.get(LocalCache.java:3932)
	at com.google.common.cache.LocalCache$LocalManualCache.get(LocalCache.java:4721)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalog.getCachedPlan(SessionCatalog.scala:171)
	at org.apache.spark.sql.execution.datasources.FindDataSourceTable.org$apache$spark$sql$execution$datasources$FindDataSourceTable$$readDataSourceTable(DataSourceStrategy.scala:267)
	at org.apache.spark.sql.execution.datasources.FindDataSourceTable$$anonfun$apply0$1.applyOrElse(DataSourceStrategy.scala:351)
	at org.apache.spark.sql.execution.datasources.FindDataSourceTable$$anonfun$apply0$1.applyOrElse(DataSourceStrategy.scala:321)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$2(AnalysisHelper.scala:112)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:80)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$1(AnalysisHelper.scala:112)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:216)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown(AnalysisHelper.scala:110)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown$(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$4(AnalysisHelper.scala:117)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$mapChildren$1(TreeNode.scala:412)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:250)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:410)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:363)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$1(AnalysisHelper.scala:117)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:216)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown(AnalysisHelper.scala:110)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown$(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$4(AnalysisHelper.scala:117)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$mapChildren$1(TreeNode.scala:412)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:250)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:410)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:363)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDo

##Data Cleansing
###Selecting the Right Data

In [0]:
retail_enriched_df.printSchema()

root
-- InvoiceNo: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: string (nullable = true)
-- InvoiceDate: string (nullable = true)
-- UnitPrice: string (nullable = true)
-- CustomerID: string (nullable = true)
-- Country: string (nullable = true)
-- CountryCode: string (nullable = true)
-- ISO2Code: string (nullable = true)
-- ISO3Code: string (nullable = true)
-- CountryName: string (nullable = true)
-- GeoShape: string (nullable = true)
-- RetailIDx: long (nullable = false)
-- age: string (nullable = true)
-- workclass: string (nullable = true)
-- fnlwgt: string (nullable = true)
-- education: string (nullable = true)
-- education-num: string (nullable = true)
-- marital-status: string (nullable = true)
-- occupation: string (nullable = true)
-- relationship: string (nullable = true)
-- race: string (nullable = true)
-- gender: string (nullable = true)
-- capital-gain: string (nullable = true)
-- capital-loss: string (nullable = true)
-- hours-per-week: string (nullable = true)
-- native-country: string (nullable = true)
-- idx: long (nullable = true)
-- IncomeIDx: long (nullable = true)

In [0]:
 retail_clean_df = (retail_enriched_df
                    .drop("Country")
                    .drop("ISO2Code")
                    .drop("ISO3Code")
                    .drop("RetailIDx")
                    .drop("idx")
                    .drop("IncomeIDx")
                   )

In [0]:
(retail_clean_df
   .select("InvoiceNo", "InvoiceDate", "StockCode")
   .groupBy("InvoiceNo", "InvoiceDate", "StockCode")
   .count()
   .show()
)

+---------+--------------+---------+-----+
InvoiceNo| InvoiceDate|StockCode|count|
+---------+--------------+---------+-----+
 536373|01/12/10 09:02| 85123A| 2|
 536382|01/12/10 09:45| 22798| 2|
 536387|01/12/10 09:58| 21731| 2|
 536388|01/12/10 09:59| 21755| 2|
 536390|01/12/10 10:19| 22960| 2|
 536437|01/12/10 12:12| 17021| 2|
 536464|01/12/10 12:23| 22956| 2|
 536464|01/12/10 12:23| 21121| 2|
 536520|01/12/10 12:43| 71406C| 2|
 536526|01/12/10 12:58| 22173| 2|
 536531|01/12/10 13:23| 22819| 2|
 536569|01/12/10 15:35| 22816| 2|
 536578|01/12/10 16:15| 21484| 2|
 536592|01/12/10 17:06| 20735| 2|
 536592|01/12/10 17:06| 84970L| 2|
 536592|01/12/10 17:06| 85008| 2|
 536592|01/12/10 17:06| 85136C| 2|
 536634|02/12/10 11:21| 22170| 2|
 536635|02/12/10 11:22| 21429| 2|
 536637|02/12/10 11:35| 72818| 2|
+---------+--------------+---------+-----+
only showing top 20 rows

In [0]:
(retail_clean_df
   .select("*")
   .where("InvoiceNo in ('536373', '536382', '536387') AND StockCode in ('85123A', '22798', '21731')")
   .display()
)

InvoiceNo StockCode Description Quantity InvoiceDate UnitPrice CustomerID CountryCode CountryName GeoShape age workclass fnlwgt education education-num marital-status occupation relationship race gender capital-gain capital-loss hours-per-week native-country 536373 85123A WHITE HANGING HEART T-LIGHT HOLDER 6 01/12/10 09:02 2.55 17850 EN United Kingdom "{""type"": ""MultiPolygon"", ""coordinates"": [[[[-6.287505663, 49.91400788], [-6.309152799, 49.913641669], [-6.298817512, 49.935492255], [-6.284169075, 49.922756252], [-6.287505663, 49.91400788]]], [[[-6.326486783, 49.94920482], [-6.332508918, 49.941717841], [-6.341216601, 49.944769598], [-6.34894772, 49.964341539], [-6.328033007, 49.956610419], [-6.326486783, 49.94920482]]], [[[-1.295765754, 50.773179429], [-1.144276496, 50.733791408], [-1.109120246, 50.732855536], [-1.097035286, 50.726955471], [-1.097035286, 50.713324286], [-1.082753059, 50.713324286], [-1.062326627, 50.692816473], [-1.062326627, 50.685288804], [-1.128651496, 50.666449286], [-1.156361457, 50.650783596], [-1.164702929, 50.640936591], [-1.168446418, 50.603461005], [-1.2792048820000002, 50.582505601], [-1.315541145, 50.589178778], [-1.485951301, 50.669338283], [-1.535145637, 50.669378973], [-1.5694880850000001, 50.658026434], [-1.5453181630000001, 50.686102606], [-1.505930142, 50.707709052], [-1.418690559, 50.733791408], [-1.425363736, 50.720119533], [-1.400868293, 50.721991278], [-1.343495246, 50.761053778], [-1.295765754, 50.773179429]]], [[[-0.9388321610000001, 50.788397528], [-0.9388321610000001, 50.780951239], [-1.02074134, 50.788397528], [-1.02074134, 50.795152085], [-0.998687304, 50.80101146], [-0.972564257, 50.829779364], [-0.9598689440000001, 50.829331773], [-0.947824674, 50.817816473], [-0.9598689440000001, 50.802069403], [-0.9388321610000001, 50.788397528]]], [[[-4.663482226, 51.164862372], [-4.670480924, 51.16010163], [-4.678537564, 51.167059637], [-4.671213345, 51.195054429], [-4.663482226, 51.164862372]]], [[[0.93637129, 51.37103913], [0.9114689460000001, 51.364406643], [0.794769727, 51.370550848], [0.757578972, 51.386419989], [0.7338973320000001, 51.412176825], [0.7468367850000001, 51.439642645], [0.7669376960000001, 51.447943427], [0.9005639980000001, 51.417547919], [0.93246504, 51.397691148], [0.9421492850000001, 51.38300202], [0.93637129, 51.37103913]]], [[[-4.655669726, 53.319769598], [-4.62629147, 53.320868231], [-4.590443489, 53.297552802], [-4.558461067, 53.252752997], [-4.56362871, 53.241929429], [-4.602406379, 53.253078518], [-4.632557746, 53.282945054], [-4.676665819, 53.28510163], [-4.693959114, 53.302801825], [-4.693959114, 53.309637762], [-4.667307095, 53.323919989], [-4.655669726, 53.319769598]]], [[[-4.203236457, 53.298488674], [-4.142486132, 53.302801825], [-4.121978319, 53.317084052], [-4.042876757, 53.302801825], [-4.098052538, 53.25014883], [-4.195546028, 53.220282294], [-4.220366991, 53.201483466], [-4.222035286, 53.182114976], [-4.280140754, 53.154974677], [-4.354359504, 53.129950262], [-4.392404752, 53.138332424], [-4.412831184, 53.132066148], [-4.411976692, 53.155585028], [-4.385121223, 53.182440497], [-4.410878059, 53.182196356], [-4.447580533, 53.159409898], [-4.48167884, 53.17987702], [-4.495350715, 53.17987702], [-4.499623176, 53.211249091], [-4.571034309, 53.275458075], [-4.558664517, 53.3633487], [-4.571034309, 53.392767645], [-4.425363736, 53.426703192], [-4.324696418, 53.419501044], [-4.276722786, 53.401027736], [-4.26691647, 53.368963934], [-4.234730598, 53.358058986], [-4.235178189, 53.34007396], [-4.214833137, 53.323919989], [-4.203236457, 53.298488674]]], [[[-6.440012174, 55.236029364], [-6.415923632, 55.231878973], [-6.378163215, 55.24624258], [-6.239816861, 55.208319403], [-6.160023567, 55.221909898], [-6.120838996, 55.217840887], [-6.047230598, 55.178534247], [-6.037098762, 55.16791413], [-6.040638801, 55.112046617], [-6.062082486, 55.07412344], [-6.054351366, 55.065130927], [-6.003529426, 55.063177802], [-5.975738085, 55.053045966], [-5.968251106, 55.04385000

In [0]:
retail_nodupe = retail_clean_df.drop_duplicates(["InvoiceNo", "InvoiceDate", "StockCode"])

In [0]:
(retail_nodupe
   .select("InvoiceNo", "InvoiceDate", "StockCode")
   .groupBy("InvoiceNo", "InvoiceDate", "StockCode")
   .count()
   .where("count > 1")
   .show()
)

+---------+-----------+---------+-----+
InvoiceNo|InvoiceDate|StockCode|count|
+---------+-----------+---------+-----+
+---------+-----------+---------+-----+

In [0]:
retail_nodupe.printSchema()

root
-- InvoiceNo: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: string (nullable = true)
-- InvoiceDate: string (nullable = true)
-- UnitPrice: string (nullable = true)
-- CustomerID: string (nullable = true)
-- CountryCode: string (nullable = true)
-- CountryName: string (nullable = true)
-- GeoShape: string (nullable = true)
-- age: string (nullable = true)
-- workclass: string (nullable = true)
-- fnlwgt: string (nullable = true)
-- education: string (nullable = true)
-- education-num: string (nullable = true)
-- marital-status: string (nullable = true)
-- occupation: string (nullable = true)
-- relationship: string (nullable = true)
-- race: string (nullable = true)
-- gender: string (nullable = true)
-- capital-gain: string (nullable = true)
-- capital-loss: string (nullable = true)
-- hours-per-week: string (nullable = true)
-- native-country: string (nullable = true)

In [0]:
retail_final_df = (retail_nodupe.selectExpr(
                                 "InvoiceNo AS invoice_num", "StockCode AS stock_code", "description AS description",
                                    "Quantity AS quantity", "InvoiceDate AS invoice_date", 
                                    "CAST(UnitPrice AS DOUBLE) AS unit_price", "CustomerID AS customer_id",
                                    "CountryCode AS country_code", "CountryName AS country_name", "GeoShape AS geo_shape",
                                    "age", "workclass AS work_class", "fnlwgt AS final_weight", "education", 
                                    "'education-num' AS education_num", "'marital-status' AS marital_status", 
                                    "occupation", "relationship", "race", "gender", "'capital-gain' AS capital_gain",
                                    "'capital-loss' AS capital_loss", "'hours-per-week' AS hours_per_week",
                                    "'native-country' AS native_country")
                  )
#retail_final_df.display()

In [0]:
retail_final_df.count()

Out[36]: 1021433

In [0]:
spark.conf.set("spark.sql.adaptive.enabled",True)

Limiting the write operation to 10K rows because of Databricks Community hard limit of 10GB storage space.

In [0]:
retail_final_df.limit(10000).write.format("delta").save("dbfs:/FileStore/shared_uploads/delta/retail_silver.delta")

In [0]:
%fs ls dbfs:/FileStore/shared_uploads/delta/retail_silver.delta

path,name,size
dbfs:/FileStore/shared_uploads/delta/retail_silver.delta/_delta_log/,_delta_log/,0
dbfs:/FileStore/shared_uploads/delta/retail_silver.delta/part-00000-1bc4c9e3-7fae-4ff7-9289-7f6e9e4b65fb-c000.snappy.parquet,part-00000-1bc4c9e3-7fae-4ff7-9289-7f6e9e4b65fb-c000.snappy.parquet,351588670


####Clean-up 
**Databricks Community Edition** has a limitation of 10000 files and 10 GB of storage in DBFS.<br>
So it is prudent to clean-up any intermediate datasets created on DBFS that we do not intent to use at a later time.

In [0]:
%fs rm -r dbfs:/FileStore/shared_uploads/delta

res3: Boolean = true

In [0]:
%sql 
DROP TABLE author_salary;

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Table or view not found: author_salary;
	at org.apache.spark.sql.execution.command.DropTableCommand.run(ddl.scala:252)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.executeCollect(commands.scala:79)
	at org.apache.spark.sql.Dataset.$anonfun$logicalPlan$1(Dataset.scala:234)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:3728)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$5(SQLExecution.scala:116)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:248)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$1(SQLExecution.scala:101)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:841)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:77)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:198)
	at org.apache.spark.sql.Dataset.withAction(Dataset.scala:3726)
	at org.apache.spark.sql.Dataset. (Dataset.scala:234)
	at org.apache.spark.sql.Dataset$.$anonfun$ofRows$2(Dataset.scala:104)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:841)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:101)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:676)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:841)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:671)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:672)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:91)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:231)
	at scala.collection.immutable.List.map(List.scala:298)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:37)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:144)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$11(DriverLocal.scala:451)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:235)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:232)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:49)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:277)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:270)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:49)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:428)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:690)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:682)
	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:523)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:635)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:428)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:371)
	at com.